In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV, LogisticRegression
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN

smoten_noncontin = pd.read_csv('smoten_noncontin.csv', index_col=False)

pd.set_option("display.max_columns", None)


def classifyScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('coef_')
    print(clf[1].coef_)
    print('intercept_')
    print(clf[1].intercept_)
    print('fit')
    print(clf.fit(X_train, y_train).n_features_in_, clf.fit(X_train, y_train).feature_names_in_)
    print('decision function \n', clf[1].decision_function(X_test))
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print('\n')
    
    
print('### SMOTEN median impute ###')
print()

X = pd.concat([smoten_noncontin['sofa_cardiovascular'], smoten_noncontin['sofa_cns'], smoten_noncontin['gender']], axis=1)

y = smoten_noncontin['outcome']

print('Logistic Regression lbfgs')

logRegress = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, 
                   fit_intercept=True, intercept_scaling=1, class_weight=None, 
                   random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', 
                   verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

print('lbfgs StandardScaler')
print()
classifyScale_coef(StandardScaler(), logRegress, 5, X, y)

print('lbfgs RobustScaler')
classifyScale_coef(RobustScaler(), logRegress, 5, X, y)


### SMOTEN median impute ###

Logistic Regression lbfgs
lbfgs StandardScaler

accuracy of each fold - [0.6909129692832765]
Avg accuracy : 0.1381825938566553
score 
 0.69597909556314
coef_
[[0.11580257 1.02896854 0.1503527 ]]
intercept_
[0.0421931]
fit
3 ['sofa_cardiovascular' 'sofa_cns' 'gender']
decision function 
 [0.30834837 1.33731691 0.19254579 ... 1.33731691 0.30834837 3.24490128]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.66      0.79      0.72      6980
 Intubation True       0.74      0.59      0.66      7084

        accuracy                           0.69     14064
       macro avg       0.70      0.69      0.69     14064
    weighted avg       0.70      0.69      0.69     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.69


lbfgs RobustScaler
accuracy of each fold - [0.6967434584755404]
Avg accuracy : 0.13934869169510808
score 
 0.694521473265074
coef_
[[0.11435501 1.54503824 0.29610857]]
intercept_
[-0.190

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
